In [19]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 12, 6
from datetime import datetime
import re
import math
from sklearn.metrics import make_scorer
from sklearn import preprocessing
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_predict

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [20]:
d = pd.read_csv('tmp/engineered_features_2.csv')
del d['Date']
del d['DateYear']
del d['DateMonth']
del d['DateDay']
len(d)

1058297

In [21]:
def rmspe_func(y_true, y_pred, **kwargs):
    if len(y_pred) != len(y_true):
        raise Exception("different length series passed to scorer function")
    d = DataFrame({'yt': y_true, 'yp': y_pred})
    d = d[d.yt != 0]
    return np.sqrt((1.0/len(d.yt))*(np.square(((d.yt - d.yp)/d.yt)).sum()))
print rmspe_func(Series([2.0, 2.0, 0.0, 0.0]), Series([1.0, 5.0, 100.0, 0.0]))
print rmspe_func(Series([2.0, 2.0, 0.0, 0.0]), Series([0.0, 0.0, 0.0, 0.0]))
rmspe = make_scorer(rmspe_func, greater_is_better=False)

1.11803398875
1.0


In [22]:
d = d[pd.notnull(d.Sales)]
len(d)

1017209

In [23]:
#d = d.iloc[np.random.permutation(len(d))]
lim_tr = int(0.4 * len(d))
tr = d[:lim_tr]
trl = tr.Sales
trl = trl - tr.StoreDaySales_50
del tr['Sales']
lim_ho = int(0.45 * len(d))
ho = d[lim_tr:lim_ho]
hol = ho.Sales
hol = hol - ho.StoreDaySales_50
del ho['Sales']
del d

In [24]:
def eval_pred(p):
    return rmspe_func(ho.StoreDaySales_50 + hol, ho.StoreDaySales_50 + p)
print eval_pred(zeros_like(hol))

0.228500905309


In [25]:
def eval_model(m):
    m.fit(tr, trl)
    p = m.predict(ho)
    return eval_pred(p)

In [38]:
eval_model(xgb.XGBRegressor(
        nthread=6,
        n_estimators=100,
        max_depth=10,
        learning_rate=0.01,
        min_child_weight=1,
        objective='reg:linear'))

0.15459191219904012

In [40]:
eval_model(xgb.XGBRegressor(
        nthread=6,
        n_estimators=100,
        max_depth=10,
        learning_rate=0.08,
        min_child_weight=1,
        objective='reg:linear'))

0.13885236471279822

In [41]:
eval_model(xgb.XGBRegressor(
        nthread=6,
        n_estimators=100,
        max_depth=10,
        learning_rate=0.05,
        min_child_weight=1,
        objective='reg:linear'))

0.13799738773885506

In [42]:
eval_model(xgb.XGBRegressor(
        nthread=6,
        n_estimators=600,
        max_depth=12,
        learning_rate=0.05,
        min_child_weight=5,
        objective='reg:linear'))

0.14994527877419181

In [43]:
eval_model(xgb.XGBRegressor(
        nthread=6,
        n_estimators=400,
        max_depth=12,
        learning_rate=0.05,
        min_child_weight=10,
        objective='reg:linear'))

0.14639459369150173

In [44]:
d = pd.read_csv('tmp/engineered_features_2.csv')
del d['Date']
del d['DateYear']
del d['Id']
tr = d[pd.notnull(d.Sales)]
trl = tr.Sales
trl = trl - tr.StoreDaySales_50
del tr['Sales']
te = d[pd.isnull(d.Sales)]
del te['Sales']

In [45]:
m = xgb.XGBRegressor(
        nthread=6,
        n_estimators=100,
        max_depth=11,
        learning_rate=0.05,
        min_child_weight=1,
        objective='reg:linear')
m.fit(tr, trl)
exgb_te = m.predict(te)
exgb_te = exgb_te + te.StoreDaySales_50
sub = DataFrame({'Id': np.arange(1, len(xgb_te)+1), 'Sales': xgb_te})
sub.to_csv('output/s_exgb_01.csv', index=False)